# Import Libraries and Read in Data

In [ ]:
import datetime
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from geopy.distance import geodesic
from geopy.distance import distance

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
import warnings
warnings.filterwarnings("ignore")

# Mount drive so that we can access google drive files
from google.colab import drive
drive.mount('/drive')

cab_data = pd.read_excel('/drive/My Drive/Colab Notebooks/MMA 830 - Pricing Analytics/cab_rides.xlsx',sheet_name="cab_rides")

Mounted at /drive


# Pre-Processing

Drop nulls

In [ ]:
cab_data.isnull().sum()
cab_data = cab_data.dropna()

In [ ]:
cab_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637976 entries, 0 to 693070
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   distance          637976 non-null  float64       
 1   cost_per_dist     637976 non-null  float64       
 2   cab_type          637976 non-null  object        
 3   time_stamp        637976 non-null  int64         
 4   Date              637976 non-null  datetime64[ns]
 5   destination       637976 non-null  object        
 6   source            637976 non-null  object        
 7   price             637976 non-null  float64       
 8   surge_multiplier  637976 non-null  float64       
 9   id                637976 non-null  object        
 10  product_id        637976 non-null  object        
 11  name              637976 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 63.3+ MB


Extract date and time features from pickup datetime column so we can then use them to identify daytime vs nightime or weekday vs weekend day

In [ ]:
# Identify if date falls on weekend or weekday
cab_data['day'] = cab_data['Date'].dt.dayofweek # <-- this outputs a number based on the day (0=monday,6=sunday)
cab_data['day_type'] = np.where(cab_data['day'].isin([5,6]),'weekend', "weekday") # <-- this uses those numbers to group each day

# Convert date to string and them strip the time from it, we will use this to group by later on
cab_data['date_string'] = cab_data['Date'].astype(str)
cab_data['date_clean'] = cab_data['date_string'].str[:-13].str.strip()

# Extract time from date_string column, identify the hour (in 24h clock)
cab_data['time_string'] = cab_data['date_string'].str[-13:-4].str.strip()
cab_data['hour'] = cab_data['time_string'].str[:2].astype(int)

# Group data based on hour to identify if it is during the DAY or EVENING
cab_data['time_type'] = np.where((cab_data['hour'] >= 8) & (cab_data['hour'] < 17),'Day', "Evening")


# Create combined segment column to get unique segments
cab_data['segment'] = cab_data['day_type']+"_"+cab_data['time_type']

Filter data to just one district, one car type, and uber only trips

In [ ]:
#cab_data = cab_data[cab_data['source'] == 'Financial District']
cab_data = cab_data[cab_data['name'] == 'UberX']
cab_data = cab_data[cab_data['cab_type'] == 'Uber']

In [ ]:
cab_data

,distance,cost_per_dist,cab_type,time_stamp,Date,destination,source,price,surge_multiplier,id,product_id,name,day,day_type,date_string,date_clean,time_string,hour,time_type,segment
14,1.11,6.756757,Uber,1544728503935,2018-12-13 19:15:03.935,West End,North End,7.5,1.0,357559cb-8c58-4278-a41a-e33b2e0997a3,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,3,weekday,2018-12-13 19:15:03.935,2018-12-13,19:15:03,19,Evening,weekday_Evening
28,2.48,3.427419,Uber,1544697611531,2018-12-13 10:40:11.531,South Station,Beacon Hill,8.5,1.0,4149295f-4534-4712-b2cf-160308528c75,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,3,weekday,2018-12-13 10:40:11.531,2018-12-13,10:40:11,10,Day,weekday_Day
41,2.94,3.231293,Uber,1544789105468,2018-12-14 12:05:05.468,Fenway,North Station,9.5,1.0,c05d4e09-3f00-43cf-a0d8-6f38c0f4d04a,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,4,weekday,2018-12-14 12:05:05.468,2018-12-14,12:05:05,12,Day,weekday_Day
64,1.16,8.189655,Uber,1543325422263,2018-11-27 13:30:22.263,West End,North End,9.5,1.0,c9dee6ef-eca0-43ba-9bf9-ec00a6638164,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,1,weekday,2018-11-27 13:30:22.263,2018-11-27,13:30:22,13,Day,weekday_Day
68,2.67,3.558052,Uber,1543286541956,2018-11-27 02:42:21.956,Beacon Hill,North End,9.5,1.0,5ed9f6ab-4582-45c6-a1de-5602e911c3bb,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,1,weekday,2018-11-27 02:42:21.956,2018-11-27,02:42:21,2,Evening,weekday_Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693008,7.36,2.173913,Uber,1544724903573,2018-12-13 18:15:03.573,North End,Back Bay,16.0,1.0,c20c7f8f-036c-41ec-9f08-dd19576cc324,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,3,weekday,2018-12-13 18:15:03.573,2018-12-13,18:15:03,18,Evening,weekday_Evening
693022,2.50,6.400000,Uber,1543807682279,2018-12-03 03:28:02.279,Beacon Hill,South Station,16.0,1.0,0572270b-0dfd-432c-b9d6-11a22c43954b,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,0,weekday,2018-12-03 03:28:02.279,2018-12-03,03:28:02,3,Evening,weekday_Evening
693052,0.91,7.692308,Uber,1543456028123,2018-11-29 01:47:08.123,Beacon Hill,Haymarket Square,7.0,1.0,aba6db9d-6b6f-4bb2-b9dd-2ceb1fbd1061,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,3,weekday,2018-11-29 01:47:08.123,2018-11-29,01:47:08,1,Evening,weekday_Evening
693061,1.61,9.316770,Uber,1543728484149,2018-12-02 05:28:04.149,Haymarket Square,Theatre District,15.0,1.0,54362c72-124b-47e8-8d07-62a07cfff624,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,6,weekend,2018-12-02 05:28:04.149,2018-12-02,05:28:04,5,Evening,weekend_Evening


# Group by Day for Each Segment

In [ ]:
# Get average price per mile or km for each day and segment
avg_prices = cab_data.groupby(['segment','date_clean'])['cost_per_dist'].mean().reset_index()

# Get number of trips for each day and segment
demands = cab_data.groupby(['segment','date_clean']).size().reset_index()

data_to_model = avg_prices.merge(demands, how='left', left_on=['segment','date_clean'], right_on=['segment','date_clean'])

In [ ]:
data_to_model = data_to_model.rename(columns={"segment":"segment", 
                                              "date_clean":"date",
                                              "cost_per_dist":"mean_cost_per_dist",
                                              0:"demand"})

data_to_model['revenue_per_dist'] = data_to_model['mean_cost_per_dist'] * data_to_model['demand']
data_to_model = data_to_model.sort_values(by=['segment','mean_cost_per_dist'],ascending=[True,False])

In [ ]:
data_to_model

,segment,date,mean_cost_per_dist,demand,revenue_per_dist
4,weekday_Day,2018-11-30,6.129536,1360,8336.169460
9,weekday_Day,2018-12-18,6.042776,1380,8339.031267
6,weekday_Day,2018-12-13,6.041934,1396,8434.540169
0,weekday_Day,2018-11-26,6.019102,1128,6789.547235
7,weekday_Day,2018-12-14,5.988717,1293,7743.410984
5,weekday_Day,2018-12-03,5.980714,1285,7685.217328
1,weekday_Day,2018-11-27,5.933366,2212,13124.605729
2,weekday_Day,2018-11-28,5.905641,3092,18260.240929
8,weekday_Day,2018-12-17,5.736833,1383,7934.040027
3,weekday_Day,2018-11-29,5.667998,1339,7589.449676


In [ ]:
# get unique segments
segments = data_to_model['segment'].unique()

# for each segment...
for segment in segments:

  # filter to get only this segment's data
  segment_data = data_to_model[data_to_model['segment'] == segment]

  # Separate prices and demand
  x_prices = segment_data[['mean_cost_per_dist']]
  y_demands = segment_data[['demand']]

  # fit linear model
  segment_model = LinearRegression().fit(x_prices, y_demands)
  model_coef = segment_model.coef_[0][0]
  model_intercept = segment_model.intercept_[0]
  model_r2 = segment_model.score(x_prices, y_demands)

  print('------------------------------------')
  print('processing segment...',segment)
  print('------------------')
  print("Coefficient:",model_coef)
  print("Intercept:",model_intercept)
  print("R-squared:",model_r2)

  # Use coefficient and intercept to predict demand and calculate predicted revenue
  segment_data['demand_prediction'] = model_intercept + model_coef * segment_data['mean_cost_per_dist']
  segment_data['revenue_prediction'] = segment_data["mean_cost_per_dist"] * segment_data["demand_prediction"]

  actual_revenue = round(segment_data['revenue_per_dist'].sum(),2)
  predict_revenue = round(segment_data['revenue_prediction'].sum(),2)
  change_in_revenue = predict_revenue - actual_revenue
  
  print('\n')
  print("Actual Revenue per Mile: $"+str(actual_revenue))
  print("Predicted Revenue per Mile: $"+str(predict_revenue))
  print("Change in Revenue: $"+str(change_in_revenue))
  print('\n')

  def custom_solver():
    output = pd.DataFrame()
    for price in np.arange(0.00,50.00,0.01):
      segment_data['solver_demand_pred'] = model_intercept + model_coef * price
      segment_data['solver_revenue_pred'] = price * segment_data["solver_demand_pred"]
      revenue = round(segment_data['solver_revenue_pred'].sum(),2)

      results = pd.DataFrame({"Price":[price],"Revenue":[revenue]})

      output = pd.concat([output, results], axis=0)

    output = output.sort_values(by=['Revenue'],ascending=False)
    optimal_price = output['Price'].iat[0]
    revenue_wOptimalPrice = output['Revenue'].iat[0]

    return optimal_price, revenue_wOptimalPrice

  optimal_price, revenue_using_optimalPrice = custom_solver()
  optimal_price = round(optimal_price,2)
  revenue_using_optimalPrice = round(revenue_using_optimalPrice,2)
  revenue_change_using_optimal = revenue_using_optimalPrice - actual_revenue

  print("Optimal price for this segment: $"+str(optimal_price))
  print("Revenue using optimal price for this segment: $"+str(revenue_using_optimalPrice))
  print("Change in revenue (using optimal vs not using optimal): $"+str(revenue_change_using_optimal))
  print("\n")


  # -----------------------------------------------------------------
  # *** FUNCTIONS TO SOLVER FOR OPTIMAL PRICE USING SCIPY ***
  # Couldn't get it to iterate through and got lazy so I created "custom_solver" above

  def sum_revenue(price):
    segment_data['solver_demand_pred'] = model_intercept + model_coef * price[0]
    segment_data['solver_revenue_pred'] = price[0] * segment_data["solver_demand_pred"]
    revenue = round(segment_data['solver_revenue_pred'].sum(),2)

    # scipy only has a minimize function so turn it to a negative value
    revenue = revenue*-1

    #print(segment_data)
    #print(revenue)

    return revenue

  def solve_optimal_price():
    prices = [3.0]
    results = minimize(sum_revenue, x0=prices, method='L-BFGS-B')
    optimal_single_price = results.x
    
    return optimal_single_price
  #optimal_price = solve_optimal_price()
  #print(optimal_price)
  # -----------------------------------------------------------------



------------------------------------
processing segment... weekday_Day
------------------
Coefficient: -510.4831133073349
Intercept: 4621.449453256523
R-squared: 0.014582837443646524


Actual Revenue per Mile: $94236.25
Predicted Revenue per Mile: $94236.25
Change in Revenue: $0.0


Optimal price for this segment: $4.53
Revenue using optimal price for this segment: $104595.93
Change in revenue (using optimal vs not using optimal): $10359.679999999993


------------------------------------
processing segment... weekday_Evening
------------------
Coefficient: -1240.1065577801066
Intercept: 9514.304476324332
R-squared: 0.05365162871175988


Actual Revenue per Mile: $148213.16
Predicted Revenue per Mile: $148213.16
Change in Revenue: $0.0


Optimal price for this segment: $3.84
Revenue using optimal price for this segment: $218985.77
Change in revenue (using optimal vs not using optimal): $70772.60999999999


------------------------------------
processing segment... weekend_Day
----------